In [3]:
# Instalación de todas las dependencias del proyecto
!pip install -q -r ../requirements.txt

# Desarrollo de una Blockchain con transactions, nodo conexion y chain replace

Creación de una clase Blockchain que contenga todos los metodos basicos:
- Creación de un nuevo bloque.
- Obtención del hash de un bloque.
- Prtocolo de concenso Proof of Work (PoW).
- Generación del hash de un bloque.
- Verificación de la validez de la Blockchain.
- Agregar una nueva transacción a la cadena de bloques.
- Agregar nuevo nodo a la cadena de bloques.
- Remplazar la Blockchain por la adecuada.

In [2]:
import sys
from pathlib import Path

# Agregar el directorio raíz del proyecto al path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Importamos nuestra blockchain
from src.blockchain import Blockchain

Aplicación Web accesible vía REST API. Mediante llamdas HTTP a la API podemos establecer una comunicacion con el uso del modulo de Flask.

Llamadas a realizar vía REST API:

- Minación de bloques: mine_block()
- Obtención de la Blockchain: get_chain()
- Comprobar estado de la Blockchain: is_valid()

In [3]:
from uuid import uuid4
from flask import Flask, jsonify, request


# Crear una aplicación web
app = Flask(__name__)

# Si se obtiene un error 500, actualizar flask y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Crear la dirección del nodo en el puerto 5000
node_address = str(uuid4()).replace('-', '')

# Creacion de la blockchain
blockchain = Blockchain()


@app.route('/mine_block', methods=['GET'])
def mine_block():
  """ Minado de un nuevo bloque """

  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  blockchain.add_transaction(sender=node_address, receiver='Ariel Duarte', amount=10)
  block = blockchain.create_block(proof, previous_hash)
  response = {
      'message': 'Succes new block has been successfully mined!',
      'index' : block['index'],
      'timestamp' : block['timestamp'],
      'proof': block['proof'],
      'previous_hash': block['previous_hash'],
      'transactions': block['transactions']
  }
  return jsonify(response), 200

@app.route('/get_chain', methods=['GET'])
def get_chain():
  """ Obtención de la Blockchain """
  response = {
      'chain': blockchain.chain,
      'length': len(blockchain.chain)
  }
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación si la Blockchain es valida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
    response = {'message': 'The blockchain is valid!'}
  else:
    response = {'message': 'The blockchain is not valid!'}
  return jsonify(response), 200

@app.route('/add_transaction', methods = ['POST'])
def add_transaction():
    """ Añadir una nueva transacción a la Blockchain """
    json = request.get_json()
    transaction_keys = ['sender', 'receiver', 'amount']
    if not all(key in json for key in transaction_keys):
        return 'Some elements of the transaction are missing', 400
    index = blockchain.add_transaction(json['sender'], json['receiver'], json['amount'])
    response = {'message': f'This transaction will be added to Block {index}'}
    return jsonify(response), 201


# Descentralizacion de la Blockchain
@app.route('/connect_node', methods = ['POST'])
def connect_node():
    """ Conexión de nuevos nodos a la Blockchain """
    json = request.get_json()
    nodes = json.get('nodes')
    if nodes is None:
        return "No node", 400
    for node in nodes:
        blockchain.add_node(node)
    response = {
        'message': 'All the nodes are now connected. The Blockchain of Recklion Coin now contains the following nodes:',
        'total_nodes': list(blockchain.nodes)
    }
    return jsonify(response), 201

@app.route('/replace_chain', methods = ['GET'])
def replace_chain():
    """ Reemplazo de la cadena por la más larga si es necesario """
    is_chain_replaced = blockchain.replace_chain()
    if is_chain_replaced:
        response = {
            'message': 'The nodes had different chains so the chain was replaced by the longest one.',
            'new_chain': blockchain.chain
        }
    else:
        response = {
            'message': 'All good. The chain is the largest one.',
            'actual_chain': blockchain.chain
        }
    return jsonify(response), 200

# Documentación de la REST API

## 1. Minar un nuevo bloque

**Descripción:** Mina un nuevo bloque en la blockchain, agregando una transacción de recompensa al minero.

**Método:** `GET`  
**Path:** `/mine_block`  
**Body:** No requiere

**Ejemplo cURL:**
```bash
curl -X GET http://localhost:5000/mine_block
```

---

## 2. Obtener la cadena completa

**Descripción:** Devuelve toda la blockchain con todos sus bloques y la longitud de la cadena.

**Método:** `GET`  
**Path:** `/get_chain`  
**Body:** No requiere

**Ejemplo cURL:**
```bash
curl -X GET http://localhost:5000/get_chain
```

---

## 3. Validar la blockchain

**Descripción:** Verifica si la blockchain es válida comprobando la integridad de todos los bloques y sus hashes.

**Método:** `GET`  
**Path:** `/is_valid`  
**Body:** No requiere

**Ejemplo cURL:**
```bash
curl -X GET http://localhost:5000/is_valid
```

---

## 4. Agregar una transacción

**Descripción:** Agrega una nueva transacción a la lista de transacciones pendientes que se incluirán en el próximo bloque minado.

**Método:** `POST`  
**Path:** `/add_transaction`  
**Body:** 
```json
{
  "sender": "dirección_del_emisor",
  "receiver": "dirección_del_receptor",
  "amount": 100
}
```

**Ejemplo cURL:**
```bash
curl -X POST http://localhost:5000/add_transaction \
  -H "Content-Type: application/json" \
  -d '{"sender": "Alice", "receiver": "Bob", "amount": 50}'
```

---

## 5. Conectar nuevos nodos

**Descripción:** Conecta nuevos nodos a la red de la blockchain para descentralizar el sistema.

**Método:** `POST`  
**Path:** `/connect_node`  
**Body:** 
```json
{
  "nodes": ["http://127.0.0.1:5001", "http://127.0.0.1:5002"]
}
```

**Ejemplo cURL:**
```bash
curl -X POST http://localhost:5000/connect_node \
  -H "Content-Type: application/json" \
  -d '{"nodes": ["http://127.0.0.1:5001", "http://127.0.0.1:5002"]}'
```

---

## 6. Reemplazar la cadena

**Descripción:** Aplica el algoritmo de consenso para reemplazar la cadena actual por la cadena más larga de la red, si es necesario.

**Método:** `GET`  
**Path:** `/replace_chain`  
**Body:** No requiere

**Ejemplo cURL:**
```bash
curl -X GET http://localhost:5000/replace_chain
```


In [8]:
# Iniciar el servidor Flask en el puerto 5000
if __name__ == '__main__':
    print("🚀 Blockchain API corriendo en: http://127.0.0.1:5000")
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

🚀 Blockchain API corriendo en: http://127.0.0.1:5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2026 13:45:20] "GET /get_chain HTTP/1.1" 200 -


In [7]:
# Iniciar el servidor Flask en el puerto 5001
if __name__ == '__main__':
    print("🚀 Blockchain API corriendo en: http://127.0.0.1:5001")
    app.run(host='127.0.0.1', port=5001, debug=False, use_reloader=False)

🚀 Blockchain API corriendo en: http://127.0.0.1:5001
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2026 13:36:33] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2026 13:36:37] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2026 13:36:39] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2026 13:42:25] "POST /add_transaction HTTP/1.1" 201 -
127.0.0.1 - - [20/Jan/2026 13:42:39] "GET /get_chain HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2026 13:43:37] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2026 13:44:37] "GET /get_chain HTTP/1.1" 200 -


In [9]:
# Iniciar el servidor Flask en el puerto 5002
if __name__ == '__main__':
    print("🚀 Blockchain API corriendo en: http://127.0.0.1:5002")
    app.run(host='127.0.0.1', port=5002, debug=False, use_reloader=False)

🚀 Blockchain API corriendo en: http://127.0.0.1:5002
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2026 13:46:11] "GET /get_chain HTTP/1.1" 200 -
